In [1]:
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
import xgboost as xgb
import random


In [2]:
# Load the new dataset
file_path = 'new_dataset/output.csv'
data = pd.read_csv(file_path)

In [3]:
# Extract the first 12 rows to determine the range of each parameter
parameter_ranges = data.iloc[:12][['Q1', 'Q2', 'EN', 'SN', 'FN', 'F', 'M']]

In [4]:
# Determine the min and max values for each parameter
min_values = parameter_ranges.min().values
max_values = parameter_ranges.max().values

In [5]:
# Correct the range for the 'F' column
F_min, F_max = 1.30e-3, 1.50e-3  # Correct range for F
min_values[5] = F_min
max_values[5] = F_max

In [6]:
# The remaining rows are the actual data for model training
data_cleaned = data.iloc[12:].reset_index(drop=True)
data_cleaned = data_cleaned.drop(columns=['Unnamed: 0', 'sim_status', 'final_neck_diameter'])
data_cleaned = data_cleaned.dropna(subset=['MAPE'])


In [7]:
# Define features and target
X = data_cleaned[['Q1', 'Q2', 'EN', 'SN', 'FN', 'F', 'M']]
y = data_cleaned['MAPE']

In [8]:
# Ensure there are no missing values in the features
X = X.dropna()



In [9]:
# Train an XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [10]:
# Define the fitness function
def fitness_function(individual):
    # Create a DataFrame with a single row containing the individual's parameters
    params = pd.DataFrame([individual], columns=['Q1', 'Q2', 'EN', 'SN', 'FN', 'F', 'M'])
    params['Q3'] = params['Q1'] ** 2  # Maintain the relationship Q3 = Q1^2
    predicted_mape = xgb_model.predict(params[['Q1', 'Q2', 'EN', 'SN', 'FN', 'F', 'M']])
    return (predicted_mape[0],)


In [11]:
# Set up the DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # We want to minimize MAPE
creator.create("Individual", list, fitness=creator.FitnessMin)

In [12]:
toolbox = base.Toolbox()


In [13]:
# Attribute generator: define how each parameter is generated
for i in range(len(min_values)):
    toolbox.register(f"attr_{i}", random.uniform, min_values[i], max_values[i])


In [14]:
# Structure initializers: define how individuals and population are created
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_0, toolbox.attr_1, toolbox.attr_2,
                  toolbox.attr_3, toolbox.attr_4, toolbox.attr_5, toolbox.attr_6), n=1)


In [15]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [16]:
# Genetic operators with constraints
def mate_and_clip(ind1, ind2):
    tools.cxBlend(ind1, ind2, alpha=0.5)
    # Enforce constraints by clipping
    for i in range(len(ind1)):
        ind1[i] = max(min(ind1[i], max_values[i]), min_values[i])
        ind2[i] = max(min(ind2[i], max_values[i]), min_values[i])
    return ind1, ind2

In [17]:
def mutate_and_clip(ind):
    tools.mutGaussian(ind, mu=0, sigma=0.1, indpb=0.2)
    # Enforce constraints by clipping
    for i in range(len(ind)):
        ind[i] = max(min(ind[i], max_values[i]), min_values[i])
    return ind,

In [18]:

toolbox.register("mate", mate_and_clip)
toolbox.register("mutate", mutate_and_clip)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitness_function)

In [19]:
# Increase population size and number of generations
population_size = 200  # Increase population size
num_generations = 100  # Increase the number of generations
cxpb = 0.5  # Crossover probability
mutpb = 0.2  # Mutation probability


In [20]:
# Create an initial population
population = toolbox.population(n=population_size)


In [21]:
# Run the genetic algorithm
algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=num_generations, verbose=True)


gen	nevals
0  	200   
1  	116   
2  	107   
3  	115   
4  	111   
5  	111   
6  	135   
7  	113   
8  	131   
9  	129   
10 	93    
11 	120   
12 	126   
13 	122   
14 	104   
15 	134   
16 	120   
17 	122   
18 	123   
19 	136   
20 	136   
21 	113   
22 	118   
23 	117   
24 	124   
25 	121   
26 	121   
27 	125   
28 	113   
29 	117   
30 	127   
31 	128   
32 	115   
33 	127   
34 	135   
35 	135   
36 	128   
37 	133   
38 	140   
39 	125   
40 	136   
41 	129   
42 	143   
43 	107   
44 	119   
45 	111   
46 	118   
47 	124   
48 	127   
49 	108   
50 	112   
51 	131   
52 	116   
53 	146   
54 	122   
55 	127   
56 	122   
57 	107   
58 	106   
59 	116   
60 	126   
61 	124   
62 	132   
63 	119   
64 	134   
65 	119   
66 	103   
67 	106   
68 	112   
69 	114   
70 	114   
71 	118   
72 	138   
73 	122   
74 	119   
75 	125   
76 	106   
77 	133   
78 	100   
79 	123   
80 	118   
81 	129   
82 	122   
83 	115   
84 	120   
85 	134   
86 	129   
87 	111   
88 	120   
89 	130   

([[1.1684649640741882,
   0.9046965120314328,
   0.2755998235574393,
   0.17036243684152821,
   0.06637464519221717,
   0.0013000162711026143,
   1070.2596153469408],
  [1.1685843494799264,
   0.9046758626176037,
   0.27418057626027975,
   0.17036312804520082,
   0.06638604640668581,
   0.001300016465632248,
   1068.4730078117998],
  [1.168565866837918,
   0.9053329207589719,
   0.2747651955944642,
   0.17006742732146585,
   0.04713598334414151,
   0.0013000362180555366,
   1071.8074946187585],
  [1.1685639043054072,
   0.9054209727395097,
   0.2746849936213086,
   0.1701041242855194,
   0.04723359018511067,
   0.0013000176098233336,
   1071.9335904253978],
  [1.1685117873464794,
   0.905324610703098,
   0.27475492096564474,
   0.1701269061514925,
   0.04709312277957695,
   0.0013000377894679443,
   1070.8481545859736],
  [1.1685614656734618,
   0.9053634901886064,
   0.27476604067944216,
   0.1700345571853642,
   0.048825805062032206,
   0.0013000359145880257,
   1069.119858351206],
 

In [22]:
# Extracting the best individuals
top_individuals = tools.selBest(population, k=10)


In [23]:
# Convert the top individuals to a DataFrame for easier analysis
top_individuals_df = pd.DataFrame(top_individuals, columns=['Q1', 'Q2', 'EN', 'SN', 'FN', 'F', 'M'])
top_individuals_df['Q3'] = top_individuals_df['Q1'] ** 2  # Maintain the relationship Q3 = Q1^2


In [24]:
# Predict their MAPE values
top_individuals_df['Predicted_MAPE'] = xgb_model.predict(top_individuals_df[['Q1', 'Q2', 'EN', 'SN', 'FN', 'F', 'M']])


In [25]:

# Sort by MAPE and display the top 10 results
top_individuals_df = top_individuals_df.sort_values(by='Predicted_MAPE')
print(top_individuals_df.head(10))


         Q1        Q2        EN        SN        FN       F            M  \
0  1.168566  0.905333  0.274765  0.170067  0.047136  0.0013  1071.807495   
1  1.168564  0.905421  0.274685  0.170104  0.047234  0.0013  1071.933590   
2  1.168512  0.905325  0.274755  0.170127  0.047093  0.0013  1070.848155   
3  1.168562  0.905419  0.274743  0.170139  0.047160  0.0013  1071.813300   
4  1.168519  0.905318  0.274686  0.170064  0.046339  0.0013  1071.833463   
5  1.168519  0.905318  0.274686  0.170064  0.046339  0.0013  1071.833463   
6  1.168477  0.905340  0.274717  0.170187  0.048112  0.0013  1067.433354   
7  1.168562  0.905419  0.274743  0.170139  0.047160  0.0013  1071.812542   
8  1.168519  0.905318  0.274686  0.170064  0.046339  0.0013  1071.833463   
9  1.168562  0.905419  0.274743  0.170139  0.047160  0.0013  1071.887746   

         Q3  Predicted_MAPE  
0  1.365546        2.296362  
1  1.365542        2.296362  
2  1.365420        2.296362  
3  1.365537        2.296362  
4  1.365437  

In [26]:
# Optionally save to a CSV for further analysis
top_individuals_df.to_csv('top_genetic_algorithm_results_constrained_1.csv', index=False)